# Day 20: Project 2 - Character-Level RNN for Text Generation

**Goal:** Build a portfolio-quality character-level text generation model.

**Time estimate:** 4-5 hours

**What you'll create:**
- A trained character-level LSTM
- Multiple generation strategies (temperature, sampling)
- Comprehensive evaluation metrics
- Professional visualizations and documentation

**Deliverables:**
- Working, trained model
- Generation examples with analysis
- Training documentation
- Portfolio-ready code

---

## Setup: Imports and Configuration

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import urllib.request
import os
from pathlib import Path
import json
from datetime import datetime

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Set seeds
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")

## Part 1: Data Preparation

### Step 1: Download and Load Dataset

In [ ]:
def download_shakespeare():
    """
    Download Shakespeare dataset or use local fallback.
    """
    filepath = "shakespeare.txt"
    
    # Try to download from Karpathy's repo
    url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
    
    if not os.path.exists(filepath):
        try:
            print(f"Downloading Shakespeare dataset...")
            urllib.request.urlretrieve(url, filepath)
            print(f"✓ Downloaded to {filepath}")
        except Exception as e:
            print(f"Download failed: {e}")
            print(f"Using sample data instead...")
            # Create small sample
            sample = """ROMEO AND JULIET

ACT I

SCENE I. A public place.

Enter SAMPSON and GREGORY, of the house of Capulet, armed with swords and bucklers.

SAMPSON: Gregory, on my word, we'll not carry coals.
GREGORY: No, for then we should be colliers.
SAMPSON: I mean, an we be in choler, we'll draw.
GREGORY: Ay, while you live, draw your neck out of collar.
SAMPSON: I strike quickly, being moved.
GREGORY: But thou art not quickly moved to strike.
SAMPSON: A dog of the house of Capulet moves me.
GREGORY: To move is to stir; and to stir is to move; but here's the point--
SAMPSON: Draw, if you dare. It is the law of our side if I say ay.
GREGORY: Nay, I'll not stir for no man's pleasure, I.
SAMPSON: This is a sorry sight.
GREGORY: True, cousin.
SAMPSON: Draw thy tool! here comes of the house of the Montagues.
GREGORY: My naked weapon is out: quarrel, I will back thee.
SAMPSON: How! turn thy back and run?
GREGORY: Fear me not.
SAMPSON: No, marry; I fear thee not.
GREGORY: Let us take the law of our sides; let them begin.
SAMPSON: I will frown as I pass by, and let them take it as they list.
GREGORY: Nay, as they dare. I will bite my thumb at them; which is a disgrace to them, if they bear it.

Enter, at a distance, ABRAHAM and BALTHASAR.

SAMPSON: Do you bite your thumb at us, sir?
ABRAHAM: Do you bite your thumb at us, sir?
SAMPSON: Is the law of our side, if I say ay?
GREGORY: No.
SAMPSON: No, sir, I do not bite my thumb at you, sir; but I bite my thumb, sir.
GREGORY: Do you quarrel, sir?
ABRAHAM: Quarrel sir! no, sir.
SAMPSON: If you do, sir, I am for you: I serve as good a man as yourself.
ABRAHAM: No better.
SAMPSON: Well, sir.

Enter BENVOLIO.

GREGORY: Say 'better': here comes one of my master's kinsmen.
SAMPSON: Yes, better, sir.
ABRAHAM: You lie.
SAMPSON: Draw, if you dare.
GREGORY: Nay, I am done.

They fight; enter, on one side, BENVOLIO; on the other, TYBALT.

BENVOLIO: Part, fools!
Put up your swords; you know not what you do.

TYBALT: What, drawn, and talk of peace! I hate the word,
As I hate hell, all Montagues, and thee:
Have at thee, coward!

They fight.

Enter, several of both houses, who join the fray; then enter Citizens, with clubs.

First Citizen: Clubs, bills, and partisans! Strike! beat them down!
Down with the Capulets! Down with the Montagues!

Enter CAPULET in his gown, and LADY CAPULET.

CAPULET: What noise is this? Give me my long sword, ho!
LADY CAPULET: A crutch, a crutch! Why call you for a sword?

Enter MONTAGUE and LADY MONTAGUE.

MONTAGUE: Thou villain Capulet,--Hold me not, let me go.
LADY MONTAGUE: Thou shalt not stir a foot to seek a foe.

Enter PRINCE, with attendants.

PRINCE: Rebellious subjects, enemies to peace,
Profaners of this neighbour-stain'd steel,--
Will they not hear? What, ho! you men, you beasts,
That quench the fire of your pernicious rage
With purple fountains issuing from your veins,
On pain of torture, from those bloody hands
Throw your mistemper'd weapons to the ground,
And hear the sentence of your moved prince.
Three civil brawls, bred of an airy word,
By thee, old Capulet, and Montague,
Have thrice disturb'd the quiet of our streets,
And made Verona's ancient citizens
Cast by their grave beseeming ornaments,
To wield old partisans, in hands as old
Curst now with age against the butt'ry peace,
Of our ill-govern'd youth. Let me be bold
Which old dissension, hot, against the peace.
BENVOLIO: And yet I do not let you know:
I have been in your service and have not
Done any thing at any time against your grace.
MONTAGUE: See what a scourge is laid upon your hate,
That heaven finds means to kill your joys with love.
And I for winking at your discords too
Have lost a brace of kinsmen: all are paid.

PRINCE: Where are the vile beginners of this fray?
BENVOLIO: O noble prince, I can discover all
The unlucky manage of this fatal brawl:
There lies the man, slain by young Romeo,
That slew thy kinsman, brave Mercutio.
LADY CAPULET: Tybalt, my cousin! O my liege, I beg
For justice, which thou, prince, must give; Romeo
Slew Tybalt; Romeo must not live.
PRINCE: And for that offence
Immediately we do exile him hence:
I have an interest in your hate's proceeding,
My blood for your rude brawls doth lie a-bleeding;
But I'll amerce you with so strong a fine
That you shall all repent the loss of mine:
I will be deaf to pleading and excuses;
Nor tears nor prayers shall purchase out abuses:
Therefore use none: let Romeo hence in haste,
Else, when he's found, that hour is his last.
Bear hence this body and attend our will:
Mercy but murders, pardoning those that kill.

Exeunt.
"""
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(sample * 20)  # Repeat to make it bigger
            print(f"✓ Created sample {filepath}")
    
    return filepath

# Download dataset
data_path = download_shakespeare()

# Load text
with open(data_path, 'r', encoding='utf-8') as f:
    text = f.read()

print(f"\nDataset loaded:")
print(f"  Total characters: {len(text):,}")
print(f"  Unique characters: {len(set(text))}")
print(f"\nFirst 200 characters:")
print(text[:200])

### Step 2: Create Character Vocabulary and Mappings

In [ ]:
# Create vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)

char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

print(f"Vocabulary size: {vocab_size}")
print(f"\nCharacters (as list):")
print(repr(''.join(chars)))

# Encode entire text
data = torch.tensor([char_to_idx[char] for char in text], dtype=torch.long)
print(f"\nEncoded data shape: {data.shape}")

### Step 3: Create PyTorch Dataset

In [ ]:
class CharacterLMDataset(Dataset):
    """
    Character-level language modeling dataset.
    Creates sequences of fixed length where target is next character.
    """
    
    def __init__(self, data, seq_len):
        """
        Args:
            data: Tensor of character indices
            seq_len: Sequence length (context window)
        """
        self.data = data
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.data) - self.seq_len
    
    def __getitem__(self, idx):
        x = self.data[idx:idx + self.seq_len]
        y = self.data[idx + self.seq_len]
        return x, y


# Configuration
seq_len = 64
batch_size = 64

# Create train/val split (90/10)
split_idx = int(0.9 * len(data))
train_data = data[:split_idx]
val_data = data[split_idx:]

# Create datasets
train_dataset = CharacterLMDataset(train_data, seq_len)
val_dataset = CharacterLMDataset(val_data, seq_len)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Train dataset: {len(train_dataset):,} sequences")
print(f"Val dataset: {len(val_dataset):,} sequences")
print(f"Sequence length: {seq_len}")
print(f"Batch size: {batch_size}")
print(f"Train batches per epoch: {len(train_loader)}")

# Example batch
x_sample, y_sample = next(iter(train_loader))
print(f"\nExample batch:")
print(f"  Input shape: {x_sample.shape}")
print(f"  Target shape: {y_sample.shape}")
print(f"  First sequence: {''.join([idx_to_char[i.item()] for i in x_sample[0]])}")
print(f"  First target: '{idx_to_char[y_sample[0].item()]}'")

## Part 2: Model Architecture

In [ ]:
class CharacterLevelLSTM(nn.Module):
    """
    Multi-layer LSTM for character-level language modeling.
    
    Architecture:
    Input (batch, seq_len) → Embedding → LSTM → Linear → Output (batch, vocab_size)
    """
    
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=512, 
                 num_layers=2, dropout=0.2):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Layers
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        """
        Args:
            x: Input tensor (batch_size, seq_len)
        
        Returns:
            logits: Predictions (batch_size, seq_len, vocab_size)
        """
        # Embedding: (batch, seq_len, embed_dim)
        x = self.embedding(x)
        x = self.dropout(x)
        
        # LSTM: (batch, seq_len, hidden_dim)
        lstm_out, _ = self.lstm(x)
        lstm_out = self.dropout(lstm_out)
        
        # Output: (batch, seq_len, vocab_size)
        logits = self.fc(lstm_out)
        
        return logits


# Initialize model
model = CharacterLevelLSTM(
    vocab_size=vocab_size,
    embed_dim=128,
    hidden_dim=512,
    num_layers=2,
    dropout=0.2
).to(device)

print(f"Model architecture:")
print(model)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## Part 3: Training Loop

### Define Training Functions

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """
    Train for one epoch.
    """
    model.train()
    total_loss = 0
    
    for batch_idx, (x, y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        
        # Forward pass
        logits = model(x)  # (batch, seq_len, vocab_size)
        
        # Compute loss (use all predictions, not just last)
        loss = criterion(logits.reshape(-1, model.vocab_size), y.reshape(-1))
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping (important for RNNs to prevent exploding gradients)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)


def evaluate(model, val_loader, criterion, device):
    """
    Evaluate model on validation set.
    
    Returns:
        loss: Cross-entropy loss
        perplexity: exp(loss)
    """
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            
            logits = model(x)
            loss = criterion(logits.reshape(-1, model.vocab_size), y.reshape(-1))
            total_loss += loss.item()
    
    avg_loss = total_loss / len(val_loader)
    perplexity = np.exp(avg_loss)
    
    return avg_loss, perplexity


print("✓ Training functions defined")

### Train Model

In [ ]:
# Training configuration
num_epochs = 30
learning_rate = 0.001
patience = 5  # Early stopping

# Initialize optimizer and loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# Training tracking
train_losses = []
val_losses = []
perplexities = []
best_val_loss = float('inf')
patience_counter = 0

print(f"Starting training...")
print(f"Device: {device}")
print("-" * 70)
print(f"{'Epoch':<8} {'Train Loss':<15} {'Val Loss':<15} {'Perplexity':<15} {'LR':<10}")
print("-" * 70)

start_time = datetime.now()

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, perplexity = evaluate(model, val_loader, criterion, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    perplexities.append(perplexity)
    
    # Learning rate scheduling
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    
    # Print progress
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"{epoch+1:<8} {train_loss:<15.4f} {val_loss:<15.4f} {perplexity:<15.4f} {current_lr:<10.6f}")
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Save best model
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'perplexity': perplexity,
        }, 'best_model.pt')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

end_time = datetime.now()
training_time = (end_time - start_time).total_seconds() / 60

print("-" * 70)
print(f"\n✓ Training complete!")
print(f"  Total time: {training_time:.1f} minutes")
print(f"  Final validation perplexity: {perplexities[-1]:.2f}")

# Load best model
checkpoint = torch.load('best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"  Loaded best model from epoch {checkpoint['epoch'] + 1}")

### Visualize Training Progress

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Loss curves
ax = axes[0, 0]
ax.plot(train_losses, label='Train Loss', linewidth=2.5, marker='o', markersize=4)
ax.plot(val_losses, label='Val Loss', linewidth=2.5, marker='s', markersize=4)
ax.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax.set_ylabel('Cross-Entropy Loss', fontsize=11, fontweight='bold')
ax.set_title('Training and Validation Loss', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Perplexity
ax = axes[0, 1]
ax.plot(perplexities, linewidth=2.5, marker='o', markersize=4, color='orange')
ax.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax.set_ylabel('Perplexity', fontsize=11, fontweight='bold')
ax.set_title('Validation Perplexity', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Log scale loss
ax = axes[1, 0]
ax.semilogy(train_losses, label='Train Loss', linewidth=2.5, marker='o', markersize=4)
ax.semilogy(val_losses, label='Val Loss', linewidth=2.5, marker='s', markersize=4)
ax.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax.set_ylabel('Loss (log scale)', fontsize=11, fontweight='bold')
ax.set_title('Loss Convergence (Log Scale)', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which='both')

# Training statistics
ax = axes[1, 1]
ax.axis('off')
stats_text = f"""
TRAINING STATISTICS

Model Configuration:
  Vocab Size: {vocab_size}
  Embedding Dim: 128
  Hidden Dim: 512
  Num Layers: 2
  Dropout: 0.2

Training Details:
  Epochs: {len(train_losses)}
  Batch Size: {batch_size}
  Sequence Length: {seq_len}
  Total Parameters: {total_params:,}

Results:
  Best Val Loss: {best_val_loss:.4f}
  Final Perplexity: {perplexities[-1]:.2f}
  Training Time: {training_time:.1f} min
"""
ax.text(0.1, 0.5, stats_text, fontsize=10, family='monospace',
        verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('training_progress.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Training progress saved to training_progress.png")

## Part 4: Text Generation

### Sampling Functions

In [ ]:
def sample_next_token(logits, temperature=1.0, top_k=None, top_p=None):
    """
    Sample next token from logits.
    
    Args:
        logits: Unnormalized scores (vocab_size,)
        temperature: Sampling temperature (1.0 = no change)
        top_k: Keep only top-k tokens
        top_p: Keep tokens with cumulative prob >= top_p
    
    Returns:
        token_idx: Sampled token index
    """
    # Apply temperature
    logits = logits / (temperature + 1e-8)
    probs = torch.softmax(logits, dim=-1)
    
    # Top-k sampling
    if top_k is not None and top_k > 0:
        topk_probs, topk_indices = torch.topk(probs, min(top_k, len(probs)))
        probs = torch.zeros_like(probs)
        probs[topk_indices] = topk_probs
        probs = probs / probs.sum()
    
    # Nucleus (top-p) sampling
    if top_p is not None and top_p < 1.0:
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        cumsum = torch.cumsum(sorted_probs, dim=-1)
        sorted_indices_to_remove = cumsum > top_p
        sorted_indices_to_remove[0] = False  # Keep at least one
        sorted_probs[sorted_indices_to_remove] = 0
        probs = torch.zeros_like(probs)
        probs[sorted_indices] = sorted_probs
        probs = probs / probs.sum()
    
    # Sample
    token_idx = torch.multinomial(probs, num_samples=1)
    return token_idx.item()


def generate_text(model, start_text, max_length, char_to_idx, idx_to_char, device,
                  temperature=1.0, top_k=None, top_p=None):
    """
    Generate text starting from start_text.
    
    Args:
        model: Trained language model
        start_text: Starting sequence (string)
        max_length: Max characters to generate
        char_to_idx, idx_to_char: Character mappings
        device: Device to run on
        temperature: Sampling temperature
        top_k: Top-k sampling
        top_p: Nucleus sampling
    
    Returns:
        generated_text: Full generated text
    """
    model.eval()
    current_text = start_text
    context = torch.tensor([char_to_idx[c] for c in start_text], 
                           dtype=torch.long).to(device)
    
    with torch.no_grad():
        for _ in range(max_length):
            # Keep only last seq_len tokens as context
            context_trimmed = context[-seq_len:].unsqueeze(0)
            
            # Forward pass
            logits = model(context_trimmed)
            next_logits = logits[0, -1, :]
            
            # Sample next token
            next_idx = sample_next_token(next_logits, temperature=temperature,
                                        top_k=top_k, top_p=top_p)
            
            # Append to context and text
            context = torch.cat([context, torch.tensor([next_idx]).to(device)])
            current_text += idx_to_char[next_idx]
    
    return current_text


print("✓ Generation functions defined")

### Generate Samples with Different Strategies

In [ ]:
# Configuration for generation
start_prompts = ["ROMEO:", "JULIET:", "The", "What"]
max_gen_length = 300

print("\n" + "="*80)
print("TEXT GENERATION SAMPLES")
print("="*80)

generated_samples = {}

# Temperature variations
print("\n" + "-"*80)
print("TEMPERATURE VARIATIONS (Temperature = Diversity)")
print("-"*80)

for temp in [0.5, 1.0, 1.5]:
    print(f"\nTemperature = {temp}:")
    generated = generate_text(model, start_prompts[0], max_gen_length,
                             char_to_idx, idx_to_char, device,
                             temperature=temp)
    generated_samples[f'temp_{temp}'] = generated
    print(generated[:250])
    print("...")

# Top-k variations
print("\n" + "-"*80)
print("TOP-K SAMPLING")
print("-"*80)

for k in [5, 10, 20]:
    print(f"\nTop-K = {k}:")
    generated = generate_text(model, start_prompts[1], max_gen_length,
                             char_to_idx, idx_to_char, device,
                             top_k=k)
    generated_samples[f'topk_{k}'] = generated
    print(generated[:250])
    print("...")

# Top-p variations
print("\n" + "-"*80)
print("NUCLEUS (TOP-P) SAMPLING")
print("-"*80)

for p in [0.5, 0.9, 0.95]:
    print(f"\nTop-P = {p}:")
    generated = generate_text(model, start_prompts[2], max_gen_length,
                             char_to_idx, idx_to_char, device,
                             top_p=p)
    generated_samples[f'topp_{p}'] = generated
    print(generated[:250])
    print("...")

### Generate from Different Prompts

In [ ]:
print("\n" + "="*80)
print("GENERATION FROM DIFFERENT PROMPTS (Temperature=1.0)")
print("="*80)

for prompt in start_prompts:
    print(f"\nPrompt: '{prompt}'")
    print("-" * 80)
    generated = generate_text(model, prompt, 500,
                             char_to_idx, idx_to_char, device,
                             temperature=1.0, top_p=0.9)
    generated_samples[f'prompt_{prompt}'] = generated
    # Print first 400 characters
    print(generated[:400])
    print("...")

## Part 5: Evaluation and Analysis

### Text Diversity Analysis

In [ ]:
def analyze_text_diversity(text, vocab_size):
    """
    Analyze generated text for diversity and quality.
    """
    # Character diversity
    unique_chars = len(set(text))
    char_entropy = -sum((text.count(c) / len(text)) * np.log(text.count(c) / len(text))
                        for c in set(text))
    
    # Token repetition (bigrams)
    bigrams = [text[i:i+2] for i in range(len(text)-1)]
    unique_bigrams = len(set(bigrams))
    bigram_diversity = unique_bigrams / len(bigrams) if bigrams else 0
    
    # Token repetition (words)
    words = text.split()
    unique_words = len(set(words))
    word_diversity = unique_words / len(words) if words else 0
    
    return {
        'unique_chars': unique_chars,
        'char_entropy': char_entropy,
        'bigram_diversity': bigram_diversity,
        'word_diversity': word_diversity,
        'text_length': len(text),
    }


# Analyze all samples
analysis_results = []

for name, text in generated_samples.items():
    stats = analyze_text_diversity(text, vocab_size)
    stats['Method'] = name
    analysis_results.append(stats)

import pandas as pd
analysis_df = pd.DataFrame(analysis_results)

# Display sorted by entropy
print("\nText Diversity Analysis:")
print(analysis_df[['Method', 'unique_chars', 'char_entropy', 'bigram_diversity', 'word_diversity']].to_string(index=False))

### Visualize Sampling Strategy Comparison

In [ ]:
# Filter for temperature comparison
temp_results = [r for r in analysis_results if 'temp_' in r['Method']]
temp_names = [r['Method'].replace('temp_', '') for r in temp_results]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Entropy by temperature
ax = axes[0]
entropies = [r['char_entropy'] for r in temp_results]
ax.bar(temp_names, entropies, color=['blue', 'green', 'red'], alpha=0.7, edgecolor='black', linewidth=2)
ax.set_ylabel('Character Entropy', fontsize=11, fontweight='bold')
ax.set_xlabel('Temperature', fontsize=11, fontweight='bold')
ax.set_title('Entropy vs Temperature', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Bigram diversity
ax = axes[1]
bigram_divs = [r['bigram_diversity'] for r in temp_results]
ax.bar(temp_names, bigram_divs, color=['blue', 'green', 'red'], alpha=0.7, edgecolor='black', linewidth=2)
ax.set_ylabel('Bigram Diversity', fontsize=11, fontweight='bold')
ax.set_xlabel('Temperature', fontsize=11, fontweight='bold')
ax.set_title('Bigram Diversity vs Temperature', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Unique characters
ax = axes[2]
unique_chars = [r['unique_chars'] for r in temp_results]
ax.bar(temp_names, unique_chars, color=['blue', 'green', 'red'], alpha=0.7, edgecolor='black', linewidth=2)
ax.set_ylabel('Unique Characters', fontsize=11, fontweight='bold')
ax.set_xlabel('Temperature', fontsize=11, fontweight='bold')
ax.set_title('Character Variety vs Temperature', fontsize=12, fontweight='bold')
ax.set_ylim([0, vocab_size * 1.1])
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('generation_analysis.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Analysis saved to generation_analysis.png")

## Part 6: Model Summary and Documentation

### Save Model and Configuration

In [ ]:
# Save model configuration and metadata
config = {
    'vocab_size': vocab_size,
    'embed_dim': 128,
    'hidden_dim': 512,
    'num_layers': 2,
    'dropout': 0.2,
    'seq_len': seq_len,
    'batch_size': batch_size,
    'num_epochs': len(train_losses),
    'best_val_loss': float(best_val_loss),
    'final_perplexity': float(perplexities[-1]),
    'training_time_minutes': training_time,
    'chars': ''.join(chars),
}

with open('model_config.json', 'w') as f:
    json.dump(config, f, indent=2)

# Save character mappings
torch.save({
    'char_to_idx': char_to_idx,
    'idx_to_char': idx_to_char,
}, 'char_mappings.pt')

print("✓ Model configuration saved to model_config.json")
print("✓ Character mappings saved to char_mappings.pt")

### Final Summary

In [ ]:
summary = f"""
╔═══════════════════════════════════════════════════════════════════════════════╗
║                   CHARACTER-LEVEL TEXT GENERATION PROJECT                     ║
║                          FINAL SUMMARY & RESULTS                              ║
╚═══════════════════════════════════════════════════════════════════════════════╝

📊 DATASET STATISTICS
  Total characters: {len(text):,}
  Unique characters (vocab): {vocab_size}
  Train sequences: {len(train_dataset):,}
  Validation sequences: {len(val_dataset):,}
  Train/Val split: 90/10

🏗️  MODEL ARCHITECTURE
  Type: Multi-layer LSTM
  Embedding dimension: 128
  Hidden dimension: 512
  Number of layers: 2
  Dropout: 0.2
  Total parameters: {total_params:,}
  Trainable parameters: {trainable_params:,}

📈 TRAINING RESULTS
  Training epochs: {len(train_losses)}
  Best validation loss: {best_val_loss:.4f}
  Final perplexity: {perplexities[-1]:.2f}
  Training time: {training_time:.1f} minutes
  Optimizer: Adam (lr=0.001)
  Learning rate schedule: StepLR (step_size=10, gamma=0.5)
  Gradient clipping: norm=1.0
  Batch size: {batch_size}

🎯 GENERATION STRATEGIES TESTED
  ✓ Temperature sampling (0.5, 1.0, 1.5)
  ✓ Top-K sampling (5, 10, 20)
  ✓ Nucleus (top-p) sampling (0.5, 0.9, 0.95)
  ✓ Multiple seed texts

📁 DELIVERABLES
  ✓ Trained model checkpoint: best_model.pt
  ✓ Model configuration: model_config.json
  ✓ Character mappings: char_mappings.pt
  ✓ Training visualization: training_progress.png
  ✓ Analysis visualization: generation_analysis.png
  ✓ Generated samples: 10+ examples with various strategies

🔍 KEY OBSERVATIONS
  • Lower temperature (0.5) → More focused, less diverse
  • Higher temperature (1.5) → More diverse, sometimes incoherent
  • Top-p sampling → Better balance of coherence and diversity
  • Model successfully learned character-level patterns
  • Generated text shows realistic structure (dialogue, punctuation)

💡 WHAT YOU LEARNED
  ✓ Character-level language modeling
  ✓ LSTM architecture and training
  ✓ Multiple text generation strategies
  ✓ Model evaluation (perplexity, diversity metrics)
  ✓ Professional code organization and documentation
  ✓ End-to-end NLP project pipeline

🚀 NEXT STEPS
  • Fine-tune on different datasets
  • Try word-level or subword tokenization
  • Experiment with Transformer architecture (Day 17+)
  • Build interactive web interface for generation
  • Compare with GPT-2/GPT-3 (Days 24-26)

╔═══════════════════════════════════════════════════════════════════════════════╗
║                     PROJECT SUCCESSFULLY COMPLETED! 🎉                       ║
╚═══════════════════════════════════════════════════════════════════════════════╝
"""

print(summary)

# Save summary
with open('PROJECT_SUMMARY.txt', 'w') as f:
    f.write(summary)

print("\n✓ Summary saved to PROJECT_SUMMARY.txt")

## Part 7: Advanced Features (Optional)

### Interactive Generation with Custom Prompts

In [ ]:
def interactive_generate(model, char_to_idx, idx_to_char, device, seq_len):
    """
    Interactive text generation interface.
    """
    print("\n" + "="*80)
    print("INTERACTIVE TEXT GENERATION")
    print("="*80)
    print("\nEnter your custom prompts and generation parameters.")
    print("(Type 'quit' to exit)\n")
    
    while True:
        prompt = input("Enter prompt: ").strip()
        if prompt.lower() == 'quit':
            break
        
        if not prompt:
            print("Prompt cannot be empty.")
            continue
        
        try:
            length = int(input("Generation length (default 300): ") or "300")
            temp = float(input("Temperature (default 1.0): ") or "1.0")
        except ValueError:
            print("Invalid input.")
            continue
        
        generated = generate_text(model, prompt, length, char_to_idx, idx_to_char,
                                 device, temperature=temp, top_p=0.9)
        
        print("\nGenerated text:")
        print("-" * 80)
        print(generated)
        print("-" * 80)
        print()

# Uncomment to run interactive mode:
# interactive_generate(model, char_to_idx, idx_to_char, device, seq_len)

print("✓ Interactive generation function defined")
print("  Uncomment and run: interactive_generate(model, char_to_idx, idx_to_char, device, seq_len)")

### Compare Sampling Strategies Side by Side

## Part 8: Conclusion and Key Takeaways

### What You've Built

✅ **Complete text generation system** from data to deployment  
✅ **Multi-layer LSTM** trained on character-level sequences  
✅ **Multiple generation strategies** for controlling output quality/diversity  
✅ **Professional evaluation** with metrics and visualizations  
✅ **Production-ready code** with documentation  

### Key Insights

1. **Temperature Controls Creativity**
   - Low (0.1-0.5): Focused, repetitive
   - Medium (1.0): Balanced
   - High (1.5+): Diverse, sometimes incoherent

2. **Sampling Strategies Matter**
   - Greedy: Fast but boring
   - Top-K: Reasonable, but not perfect
   - Top-P: Excellent balance (industry standard)

3. **Perplexity Measures Uncertainty**
   - Lower is better
   - Baseline: vocab_size (random)
   - Good models: 50-200

4. **Character vs Word Level**
   - Character: Handles everything, longer sequences
   - Word: Faster, semantic awareness, needs preprocessing
   - Modern: Subword (BPE, SentencePiece) - best of both

### Architecture Insights

- **LSTM vs GRU**: LSTMs more common for language modeling
- **Gradient clipping**: Essential for RNNs to prevent explosion
- **Dropout**: Helps prevent overfitting
- **Layer normalization**: Increasingly used in modern architectures

### Real-World Applications

- Code completion (GitHub Copilot)
- Chatbots and dialogue systems
- Creative writing and story generation
- Machine translation
- Question answering systems

### Portfolio Value

This project demonstrates:
- Deep learning implementation from scratch
- NLP fundamentals
- Problem-solving and debugging
- Professional code organization
- Clear communication of results

---

**Congratulations on completing Phase 2!** 🎉

You've progressed from basic neural networks (Phase 1) to understanding and implementing:
- Recurrent architectures
- Sequence modeling
- Attention mechanisms
- Transformers
- Complete NLP systems

Next up: **Phase 3 (Days 21-30)** focuses on implementing Transformers and training GPT from scratch!